<a href="https://colab.research.google.com/github/alexandre-cameron-borges/ACBorges_Clickscore_SDUDAnalytics/blob/main/ACBorges_Clickscore_SDUDAnalytics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -q 'jupyter-client<8.0.0'

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.5/133.5 kB 3.9 MB/s eta 0:00:00


In [20]:
# 1) Cloner le repo & installer les outils
!git clone https://github.com/alexandre-cameron-borges/ACBorges_Clickscore_SDUDAnalytics.git
%cd ACBorges_Clickscore_SDUDAnalytics
!pip install -q -r requirements.txt nbstripout kaggle

# 2) Nettoyer les notebooks avant commit
!nbstripout --install

# 3) Monter Google Drive
from google.colab import drive
drive.mount('/content/drive')

# 4) Configurer l’API Kaggle
!mkdir -p ~/.kaggle
!cp /content/drive/MyDrive/credentials/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# 5) Créer le dossier raw
!mkdir -p /content/drive/MyDrive/data/raw

# 6) Télécharger & dézipper les jeux Kaggle
!kaggle datasets download -d amananandrai/clickbait-dataset \
    -p /content/drive/MyDrive/data/raw --unzip
!kaggle datasets download -d marius2303/ad-click-prediction-dataset \
    -p /content/drive/MyDrive/data/raw --unzip
!kaggle datasets download -d souvik1618/ad-click-prediction-dataset \
    -p /content/drive/MyDrive/data/raw --unzip

# 7) Télécharger + unzip Webis Clickbait Corpus 2017
!wget -q -O /content/drive/MyDrive/data/raw/clickbait17-train-170630.zip "https://zenodo.org/record/5530410/files/clickbait17-train-170630.zip?download=1"
!unzip -q /content/drive/MyDrive/data/raw/clickbait17-train-170630.zip -d /content/drive/MyDrive/data/raw/webis_clickbait
!rm /content/drive/MyDrive/data/raw/clickbait17-train-170630.zip


Cloning into 'ACBorges_Clickscore_SDUDAnalytics'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 12 (delta 1), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (12/12), done.
Resolving deltas: 100% (1/1), done.
/content/ACBorges_Clickscore_SDUDAnalytics/ACBorges_Clickscore_SDUDAnalytics/ACBorges_Clickscore_SDUDAnalytics/ACBorges_Clickscore_SDUDAnalytics/ACBorges_Clickscore_SDUDAnalytics/ACBorges_Clickscore_SDUDAnalytics
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Dataset URL: https://www.kaggle.com/datasets/amananandrai/clickbait-dataset
License(s): unknown
Dataset URL: https://www.kaggle.com/datasets/marius2303/ad-click-prediction-dataset
License(s): apache-2.0
Dataset URL: https://www.kaggle.com/datasets/souvik1618/ad-click-prediction-dataset
License(s): apache-2.0
replace /content

In [21]:
import pandas as pd

# Chemin vers le CSV amandarai clickbaitdataset dans ton Drive
pathtableclickbait_a = "/content/drive/MyDrive/data/raw/clickbait_data.csv"
tableclickbait_a = pd.read_csv(pathtableclickbait_a)

# Aperçu rapide
print("Shape :", tableclickbait_a.shape)
display(tableclickbait_a.head(8))

# Comptage des nulls et types
display(tableclickbait_a.info())
display(tableclickbait_a.isnull().sum())

# Répartition clickbait vs non-clickbait
counts = tableclickbait_a['clickbait'].value_counts(normalize=True)
print(counts)



Shape : (32000, 2)


,headline,clickbait
0,Should I Get Bings,1
1,Which TV Female Friend Group Do You Belong In,1
2,"The New ""Star Wars: The Force Awakens"" Trailer...",1
3,"This Vine Of New York On ""Celebrity Big Brothe...",1
4,A Couple Did A Stunning Photo Shoot With Their...,1
5,How To Flirt With Queer Girls Without Making A...,1
6,32 Cute Things To Distract From Your Awkward T...,1
7,If Disney Princesses Were From Florida,1


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32000 entries, 0 to 31999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   headline   32000 non-null  object
 1   clickbait  32000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 500.1+ KB


None

,0
headline,0
clickbait,0


clickbait
0    0.500031
1    0.499969
Name: proportion, dtype: float64


In [23]:
# Chemin vers le CSV marius clickbaitdataset dans ton Drive
pathtableclickbait_b = "/content/drive/MyDrive/data/raw/ad_click_dataset.csv"
tableclickbait_b = pd.read_csv(pathtableclickbait_b)

# Aperçu rapide
print("Shape :", tableclickbait_b.shape)
display(tableclickbait_b.head(8))

# Comptage des nulls et types
display(tableclickbait_b.info())
display(tableclickbait_b.isnull().sum())

# Répartition clickbait vs non-clickbait
counts = tableclickbait_b['click'].value_counts(normalize=True)
print(counts)


Shape : (10000, 9)


,id,full_name,age,gender,device_type,ad_position,browsing_history,time_of_day,click
0,670,User670,22.0,NaN,Desktop,Top,Shopping,Afternoon,1
1,3044,User3044,NaN,Male,Desktop,Top,NaN,NaN,1
2,5912,User5912,41.0,Non-Binary,NaN,Side,Education,Night,1
3,5418,User5418,34.0,Male,NaN,NaN,Entertainment,Evening,1
4,9452,User9452,39.0,Non-Binary,NaN,NaN,Social Media,Morning,0
5,5942,User5942,NaN,Non-Binary,NaN,Bottom,Social Media,Evening,1
6,7808,User7808,26.0,Female,Desktop,Top,NaN,NaN,1
7,5065,User5065,40.0,Male,Mobile,Side,NaN,Evening,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                10000 non-null  int64  
 1   full_name         10000 non-null  object 
 2   age               5234 non-null   float64
 3   gender            5307 non-null   object 
 4   device_type       8000 non-null   object 
 5   ad_position       8000 non-null   object 
 6   browsing_history  5218 non-null   object 
 7   time_of_day       8000 non-null   object 
 8   click             10000 non-null  int64  
dtypes: float64(1), int64(2), object(6)
memory usage: 703.3+ KB


None

,0
id,0
full_name,0
age,4766
gender,4693
device_type,2000
ad_position,2000
browsing_history,4782
time_of_day,2000
click,0


click
1    0.65
0    0.35
Name: proportion, dtype: float64


In [25]:
# Chemin vers le CSV souvik clickbaitdataset dans ton Drive
pathtableclickbait_c = "/content/drive/MyDrive/data/raw/advertising.csv"
tableclickbait_c = pd.read_csv(pathtableclickbait_c)

# Aperçu rapide
print("Shape :", tableclickbait_c.shape)
display(tableclickbait_c.head(8))

# Comptage des nulls et types
display(tableclickbait_c.info())
display(tableclickbait_c.isnull().sum())

# Répartition clickbait vs non-clickbait
counts = tableclickbait_c['Clicked on Ad'].value_counts(normalize=True)
print(counts)

Shape : (1000, 10)


,Daily Time Spent on Site,Age,Area Income,Daily Internet Usage,Ad Topic Line,City,Male,Country,Timestamp,Clicked on Ad
0,68.95,35,61833.90,256.09,Cloned 5thgeneration orchestration,Wrightburgh,0,Tunisia,2016-03-27 00:53:11,0
1,80.23,31,68441.85,193.77,Monitored national standardization,West Jodi,1,Nauru,2016-04-04 01:39:02,0
2,69.47,26,59785.94,236.50,Organic bottom-line service-desk,Davidton,0,San Marino,2016-03-13 20:35:42,0
3,74.15,29,54806.18,245.89,Triple-buffered reciprocal time-frame,West Terrifurt,1,Italy,2016-01-10 02:31:19,0
4,68.37,35,73889.99,225.58,Robust logistical utilization,South Manuel,0,Iceland,2016-06-03 03:36:18,0
5,59.99,23,59761.56,226.74,Sharable client-driven software,Jamieberg,1,Norway,2016-05-19 14:30:17,0
6,88.91,33,53852.85,208.36,Enhanced dedicated support,Brandonstad,0,Myanmar,2016-01-28 20:59:32,0
7,66.00,48,24593.33,131.76,Reactive local challenge,Port Jefferybury,1,Australia,2016-03-07 01:40:15,1


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 10 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Daily Time Spent on Site  1000 non-null   float64
 1   Age                       1000 non-null   int64  
 2   Area Income               1000 non-null   float64
 3   Daily Internet Usage      1000 non-null   float64
 4   Ad Topic Line             1000 non-null   object 
 5   City                      1000 non-null   object 
 6   Male                      1000 non-null   int64  
 7   Country                   1000 non-null   object 
 8   Timestamp                 1000 non-null   object 
 9   Clicked on Ad             1000 non-null   int64  
dtypes: float64(3), int64(3), object(4)
memory usage: 78.3+ KB


None

,0
Daily Time Spent on Site,0
Age,0
Area Income,0
Daily Internet Usage,0
Ad Topic Line,0
City,0
Male,0
Country,0
Timestamp,0
Clicked on Ad,0


Clicked on Ad
0    0.5
1    0.5
Name: proportion, dtype: float64


In [32]:
# 1) Chemin vers fichier webis JSONL & 2) Charger le JSONL
pathtableclickbait_d = "/content/drive/MyDrive/data/raw/webis_clickbait/clickbait17-validation-170630/truth.jsonl"
pathtableclickbait_d2 = "/content/drive/MyDrive/data/raw/webis_clickbait/clickbait17-validation-170630/instances.jsonl"
tableclickbait_d = pd.read_json(pathtableclickbait_d, lines=True)
tableclickbait_d2 = pd.read_json(pathtableclickbait_d2, lines=True)

# Aperçu rapide
print("Shape :", tableclickbait_d.shape)
display(tableclickbait_d.head(8))

print("Shape :", tableclickbait_d2.shape)
display(tableclickbait_d2.head(8))

# Comptage des nulls et types
display(tableclickbait_d.info())
display(tableclickbait_d.isnull().sum())

display(tableclickbait_d2.info())
display(tableclickbait_d2.isnull().sum())

# Répartition clickbait vs non-clickbait
counts = tableclickbait_d['truthClass'].value_counts(normalize=True)
print(counts)

Shape : (19538, 6)


,truthJudgments,truthMean,id,truthClass,truthMedian,truthMode
0,"[1.0, 1.0, 1.0, 1.0, 1.0]",1.000000,858464162594172928,clickbait,1.000000,1.000000
1,"[0.33333333330000003, 0.0, 0.33333333330000003...",0.133333,858462320779026432,no-clickbait,0.000000,0.000000
2,"[0.33333333330000003, 0.6666666666000001, 1.0,...",0.400000,858460992073863168,no-clickbait,0.333333,0.000000
3,"[0.0, 0.6666666666000001, 0.0, 0.3333333333000...",0.266667,858459539296980992,no-clickbait,0.333333,0.333333
4,"[0.0, 0.0, 0.0, 0.0, 0.0]",0.000000,858455355948384256,no-clickbait,0.000000,0.000000
5,"[0.0, 0.6666666666000001, 0.33333333330000003,...",0.200000,858453552892694528,no-clickbait,0.000000,0.000000
6,"[0.0, 0.0, 0.0, 0.0, 0.0]",0.000000,858450499921731584,no-clickbait,0.000000,0.000000
7,"[0.0, 0.0, 0.0, 0.6666666666000001, 0.0]",0.133333,858449569750896640,no-clickbait,0.000000,0.000000


Shape : (19538, 9)


,postMedia,postText,id,targetCaptions,targetParagraphs,targetTitle,postTimestamp,targetKeywords,targetDescription
0,[],[UK’s response to modern slavery leaving victi...,858462320779026432,[modern-slavery-rex.jpg],[Thousands of modern slavery victims have not ...,‘Inexcusable’ failures in UK’s response to mod...,Sat Apr 29 23:25:41 +0000 2017,"modern slavery, Department For Work And Pensio...",“Inexcusable” failures in the UK’s system for ...
1,[],[this is good],858421020331560960,"[In this July 1, 2010 file photo, Dr. Charmain...",[President Donald Trump has appointed the pro-...,Donald Trump Appoints Pro-Life Advocate as Ass...,Sat Apr 29 20:41:34 +0000 2017,"Americans United for Life, Dr. Charmaine Yoest...",President Donald Trump has appointed pro-life ...
2,[],"[The ""forgotten"" Trump roast: Relive his bruta...",858368123753435136,[President Trump will not attend this year's W...,[When the White House correspondents’ dinner i...,The ‘forgotten’ Trump roast: Relive his brutal...,Sat Apr 29 17:11:23 +0000 2017,"trump whcd, whcd, white house correspondents d...",President Trump won't be at this year's White ...
3,[],[Meet the happiest #dog in the world!],858323428260139008,"[Maru , Maru, Maru, Maru, Maru]",[Adorable is probably an understatement. This ...,"Meet The Happiest Dog In The World, Maru The H...",Sat Apr 29 14:13:46 +0000 2017,"Maru, husky, dogs, pandas, furball, instagram","The article is about Maru, a husky dog who has..."
4,[],[Tokyo's subway is shut down amid fears over a...,858283602626347008,[All nine lines of Tokyo's subway system were ...,[One of Tokyo's major subways systems says it ...,Tokyo's subway is shut down amid fears over an...,Sat Apr 29 11:35:31 +0000 2017,"Tokyo,subway,shut,fears,North,Korean,attack","The temporary suspension, which lasted ten min..."
5,[],[Ban lifted on Madrid doping laboratory],858224473597779968,"[Samples in an anti-doping laboratory, Anthony...","[Share this with, Madrid's Anti-Doping Laborat...",World Anti-Doping Agency lifts ban on Madrid l...,Sat Apr 29 07:40:34 +0000 2017,,Madrid's Anti-Doping Laboratory has its suspen...
6,[],"[Despite the ‘Yuck Factor,’ Leeches Are Big in...",858222698367000576,"[The New York Times, Basic, All Access, Home D...",[MOSCOW — They are small as physician assistan...,"Despite the ‘Yuck Factor,’ Leeches Are Big in ...",Sat Apr 29 07:33:30 +0000 2017,"Drugs (Pharmaceuticals),Medical Devices,Russia...",Russians use about 10 million leeches a year t...
7,[],[#China and #Pakistan have cemented their poli...,858191667739987968,[],[China and Pakistan have cemented their politi...,China And Pakistan Use Biryani To Take Their F...,Sat Apr 29 05:30:12 +0000 2017,"Pakistan, China, Biryani, CPEC, china pakistan...",China And Pakistan Use Biryani To Take Their F...


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19538 entries, 0 to 19537
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   truthJudgments  19538 non-null  object 
 1   truthMean       19538 non-null  float64
 2   id              19538 non-null  int64  
 3   truthClass      19538 non-null  object 
 4   truthMedian     19538 non-null  float64
 5   truthMode       19538 non-null  float64
dtypes: float64(3), int64(1), object(2)
memory usage: 916.0+ KB


...

,0
truthJudgments,0
truthMean,0
id,0
truthClass,0
truthMedian,0
truthMode,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19538 entries, 0 to 19537
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   postMedia          19538 non-null  object
 1   postText           19538 non-null  object
 2   id                 19538 non-null  int64 
 3   targetCaptions     19538 non-null  object
 4   targetParagraphs   19538 non-null  object
 5   targetTitle        19538 non-null  object
 6   postTimestamp      19538 non-null  object
 7   targetKeywords     19538 non-null  object
 8   targetDescription  19538 non-null  object
dtypes: int64(1), object(8)
memory usage: 1.3+ MB


None

,0
postMedia,0
postText,0
id,0
targetCaptions,0
targetParagraphs,0
targetTitle,0
postTimestamp,0
targetKeywords,0
targetDescription,0


truthClass
no-clickbait    0.756321
clickbait       0.243679
Name: proportion, dtype: float64
